# HashiCorp Vault Database Secrets Engine

## Value

* Reduced effort for Regulatory Compliance by Centralizating Identity & Policy for Identity AuthN/AuthZ
* Reduced Risk of Breach by Centralizating Identity & Policy for Identity AuthN/AuthZ
* Reduced Risk of Downtime due to unauthorized access
* Reduced Risk of Breach due to secrets exposure

## Database Engine - MariaDB/MySQL Dynamic Credentials
---
Dynamic Secrets are generated when they are accessed.
In other words, Dynamic Secrets do not exist until they are read.
Vault has built-in revocation mechanism.
Database Secrets engine supports:
* Cassandra
* Elasticsearch
* Influxdb
* HanaDB
* MongoDB
* MSSQL
* MySQL
* MariaDB
* PostgreSQL
* Snowflake
* Oracle

### Workflow Diagram

<img src=../../Diagrams/vault_dynamic_db_creds_seq_diagram.png>

## Business Value
We use dynamic secrets to allow applications to get dynamic creds to access databases.

---

In [ ]:
export VAULT_ADDR=http://localhost:8200
export VAULT_TOKEN=root

## Prerequisites

* Running Vault Server at localhost:8200 - see [Start Vault Server](../01-baseline/100-Setup-Vault.ipynb)

## Database Setup

### Start the database server

In [ ]:
docker pull mysql/mysql-server:5.7

In [ ]:
docker run --name workshop-mysql  \
  --rm -d \
  -p 3306:3306 \
  -e TZ=America/New_York \
  -e MYSQL_ROOT_PASSWORD=root \
  -e MYSQL_ROOT_HOST=% \
  -e MYSQL_DATABASE=my_app \
  mysql/mysql-server:5.7

Verify that your database container has started.

In [ ]:
docker ps -a

Create a bash alias for the `mysql` command.

In [ ]:
# set mysql alias
alias mysql="docker exec -it workshop-mysql mysql"

In [ ]:
#Display existing database users
mysql -u root -p'root' -e "select user from mysql.user;"

### Populate sample data

In [ ]:
#==> Create table
mysql -u root -proot -e "CREATE TABLE IF NOT EXISTS my_app.customers (
    cust_no int(11) NOT NULL AUTO_INCREMENT,
    birth_date varchar(255) NOT NULL,
    first_name varchar(255) NOT NULL,
    last_name varchar(255) NOT NULL,
    create_date varchar(255) NOT NULL,
    social_security_number varchar(255) NOT NULL,
    address varchar(255) NOT NULL,
    salary varchar(255) NOT NULL,
    PRIMARY KEY (cust_no)
) ENGINE=InnoDB;"

In [ ]:
#==> Enter data into table
mysql -u root -proot -e "INSERT IGNORE into my_app.customers VALUES
  (2, '3/14/69', 'Larry', 'Johnson', '2020-01-01T14:49:12.301977', '360-56-6750', 'Tyler, Texas', '7000000'),
  (40, '11/26/69', 'Shawn', 'Kemp', '2020-02-21T10:24:55.985726', '235-32-8091', 'Elkhart, Indiana', '15000000'),
  (34, '2/20/63', 'Charles', 'Barkley', '2019-04-09T01:10:20.548144', '531-72-1553', 'Leeds, Alabama', '9000000');
"

### Create static service account

Create static service account `dbsvc1`. This could be an existing account used by an application.

In [ ]:
mysql -u root -p'root' \
    -e "CREATE USER 'dbsvc1'@'%' IDENTIFIED BY 'dbsvc1';"
mysql -u root -p'root' \
    -e "GRANT INSERT,SELECT,UPDATE,DELETE ON my_app.* TO 'dbsvc1'@'%';"

* `IDENTIFIED BY` - sets password

#### Verify static service account

In [ ]:
#View sample data
mysql -u dbsvc1 -pdbsvc1 -e "SELECT * FROM my_app.customers"

## Vault Setup

### Create service account for Vault DB Connection

User `vaultadmin` with password `vaultadmin`.

In [ ]:
mysql -u root -proot \
    -e "CREATE USER 'vaultadmin'@'%' IDENTIFIED BY 'vaultadmin';"

mysql -u root -proot \
-e "GRANT ALL PRIVILEGES ON *.* TO 'vaultadmin'@'%' WITH GRANT OPTION;"

Verify service account user

In [ ]:
mysql -u vaultadmin -p'vaultadmin' -e "select user from mysql.user;"

### Enable Database secrets engine

In [ ]:
vault secrets enable -path=lob_a/db01 database

Confirm secrets engine has been enabled.

In [ ]:
vault secrets list

You should see:
```
Path           Type         Accessor              Description
----           ----         --------              -----------
<snip>
lob_a/db01/    database     database_fe7256b2     n/a
```

### Configure Database Secrets Engine Connection

In [ ]:
vault write lob_a/db01/config/db01 \
    plugin_name=mysql-database-plugin \
    connection_url="{{username}}:{{password}}@tcp(host.docker.internal:3306)/" \
    allowed_roles="db-user-readonly","db-user-readwrite","db-user-static" \
    username="vaultadmin" \
    password="vaultadmin"

NOTE: The username and password are templated in the "`connection_url`" string. They get their values from the "`username`" and "`password`" fields.  We do this so that reading the path "`lob_a/db01/config/db01`" will not show them.

Verify that you cannot see the password.

In [ ]:
#optional verification
vault read lob_a/db01/config/db01

### Static Roles - Create

Take over password management of user `dbsvc1` by Vault.

In [ ]:
vault write lob_a/db01/static-roles/db-user-static \
  db_name=db01 \
  rotation_period=1h \
  username=dbsvc1

The above command creates a `db-user-static` static role with database username `dbsvc1` whose password gets rotated every 3600 seconds (1 hours). The `rotation.sql` statement is passed as the rotation statement.

NOTE: For static roles, the `db_name` parameter is the database configuration name (not the database name). In this scenario, you configured `lob_a/db01/config/db01`; therefore, the `db_name` must be set to `db01`.

**Verify**: Read back the `db-user-static` role definition.

In [ ]:
vault read lob_a/db01/static-roles/db-user-static

#### Request Database credentials for static role

In [ ]:
vault read -format=json lob_a/db01/static-creds/db-user-static \
  | jq -r .data | tee pov_db_static_cred.txt

#### Verify static service account

In [ ]:
#View sample data
STATIC_ROLE_PW=$(jq -r .password pov_db_static_cred.txt)
mysql -u dbsvc1 -p${STATIC_ROLE_PW} -e "SELECT * FROM my_app.customers"

#### Manually rotate static role password

The password for the static role gets automatically rotated after a configured rotation period. Vault provides the `/database/rotate-role/<role_name>` endpoint to force an immediate password rotation.

Rotate the password for static role, "`db-user-static`".

In [ ]:
vault write -f lob_a/db01/rotate-role/db-user-static

Read the credentials to verify that the password has been rotated

In [ ]:
vault read -format=json lob_a/db01/static-creds/db-user-static | jq -r .data

The returned password should be different from previous output, and the remaining TTL has been reset to 3600 seconds.

### Create Vault DB Dynamic Roles

Create two Vault roles so applications can access credentials.

|             | default_ttl | max_ttl |
|-------------|-------------|------   |
| `db-user-readonly`  | 1h | 24h |
| `db-user-readwrite` | 5m | 24h |

In [ ]:
vault write lob_a/db01/roles/db-user-readonly \
    db_name=db01 \
    creation_statements="CREATE USER '{{name}}'@'%' IDENTIFIED BY '{{password}}';GRANT SELECT ON my_app.* TO '{{name}}'@'%';" \
    default_ttl="1h" \
    max_ttl="24h"

In [ ]:
vault write lob_a/db01/roles/db-user-readwrite \
    db_name=db01 \
    creation_statements="CREATE USER '{{name}}'@'%' IDENTIFIED BY '{{password}}';GRANT INSERT,SELECT,UPDATE,DELETE ON my_app.* TO '{{name}}'@'%';" \
    default_ttl="5m" \
    max_ttl="24h"

### Authentication and Policy Setup

Enable Userpass auth method.

In [ ]:
# Let's setup User/Pass Auth Method
vault auth enable userpass

Create Vault userpass users.

|             | password | policies | token_max_ttl | 
|-------------|-------------|------   | --- |
| `alice`  | secret | db01-readwrite-policy | 5m |
| `bob` | secret | db01-readonly-policy | 5m |

In [ ]:
# Create a few DB Users
vault write auth/userpass/users/alice \
    password=secret \
    policies=db01-readwrite-policy \
    token_max_ttl=5m
    
vault write auth/userpass/users/bob \
    password=secret \
    policies=db01-readonly-policy \
    token_max_ttl=5m

Create Vault policies: `db01-readonly-policy` and `db01-readwrite-policy`.

In [ ]:
# Now setup the Vault Policies
vault policy write db01-readonly-policy - << EOF
path "/lob_a/db01/creds/db-user-readonly" {
  capabilities = [ "read" ]
}
EOF

vault policy write db01-readwrite-policy - << EOF
path "/lob_a/db01/creds/db-user-readwrite" {
  capabilities = [ "read" ]
}
EOF

## Demo as Alice (read-write access)

Let's login as Alice. We will need to unset `VAULT_TOKEN`.

In [ ]:
# Let's login as Alice
unset VAULT_TOKEN
vault login -method=userpass username=alice password=secret

### Get Dynamic Database credential by reading from the Vault role

Confirm that Alice does NOT have access to `lob_a/db01/creds/db-user-readonly`. 

In [ ]:
vault read -format=json lob_a/db01/creds/db-user-readonly

**NOTE**: Alice's policy only allows access to `db-user-readwrite`.

Confirm that Alice does have access to `lob_a/db01/creds/db-user-readwrite`.

In [ ]:
vault read -format=json lob_a/db01/creds/db-user-readwrite \
  | tee pov_db_dyn_cred.txt

In these results, you see several thinggs:
* `lease_id` - used if you want to renew or revoke the credentials (as you will do in the next challenge).
* `username` and `password` - used to connect to the database.
* `renewable` - has the value `true`, indicating that the lifetime of the credentials can be extended using Vault's `sys/leases/renew` API endpoint.

In [ ]:
# results=$(vault read -format=json lob_a/db01/creds/db-user-readwrite)
parsed=$(jq .data[] -r pov_db_dyn_cred.txt)
export readwrite_credentials=($parsed)
echo ${readwrite_credentials[*]}

Use the last set of credentials to connect to the database server. Verify that you can see the databases on the MySQL server.

In [ ]:
mysql -u ${readwrite_credentials[1]} -p${readwrite_credentials[0]} -e "show databases;"

Expected Output
```
+--------------------+
| Database           |
+--------------------+
| information_schema |
| my_app             |
+--------------------+
```

**NOTE:** You only have 5 minutes to test this before the credentials get revoked.

Display database users. Need to use privileged user.

Note the new dynamic DB users.

In [ ]:
#Display existing database users
mysql -u root -p'root' -e "select user from mysql.user;"

You should see a user called `v-userpass-a-db-user-re-*`.

```
+----------------------------------+
| user                             |
+----------------------------------+
| dbsvc1                           |
| root                             |
| v-userpass-a-db-user-re-ITUe4SLs |
| vaultadmin                       |
| mysql.session                    |
| mysql.sys                        |
| root                             |
+----------------------------------+
```

Confirm that dynamic DB user can read the table.

In [ ]:
mysql -u ${readwrite_credentials[1]} -p${readwrite_credentials[0]} -e "select first_name from my_app.customers;"

Expected Output
```
+------------+
| first_name |
+------------+
| Larry      |
| Charles    |
| Shawn      |
+------------+
```

Confirm that dynamic DB user can NOT read from `mysql.user` table.

In [ ]:
mysql -u ${readwrite_credentials[1]} -p${readwrite_credentials[0]} -e "select user from mysql.user;"

Expected Output
```
ERROR 1045 (28000): Access denied for user 'v-userpass-a-db-user-re-ITUe4SLs'@'localhost' (using password: YES)
```

Confirm that dynamic DB user can write to `my_app.customers` table.

In [ ]:
mysql -u ${readwrite_credentials[1]} -p${readwrite_credentials[0]} -e "INSERT IGNORE into my_app.customers VALUES
  (5, '3/14/69', 'Moayad', 'Ismail', '2020-01-01T14:49:12.301977', '360-56-6750', 'Tyler, Texas', '7000000')"

Confirm that the new record is in the table.

In [ ]:
mysql -u ${readwrite_credentials[1]} -p${readwrite_credentials[0]} -e "select first_name from my_app.customers;"

## Demo as Bob (readonly access)

In [ ]:
unset VAULT_TOKEN
vault login -method=userpass username=bob password=secret

Confirm that you are logged in as `bob`.

In [ ]:
vault token lookup

### Get Dyamic Database credential by reading from the Vault role

Confirm that Bob does have access to `lob_a/db01/creds/db-user-readonly`.

In [ ]:
vault read -format=json lob_a/db01/creds/db-user-readonly

Confirm that Bob does NOT have access to `lob_a/db01/creds/db-user-readwrite`.

In [ ]:
vault read -format=json lob_a/db01/creds/db-user-readwrite

Generate dynamic DB credentials.

In [ ]:
results=$(vault read -format=json lob_a/db01/creds/db-user-readonly)
parsed=$(echo $results | jq .data[] -r)
export readonly_credentials=($parsed)
echo ${readonly_credentials[*]}

Confirm that dynamic DB user can NOT read from `mysql.user` table.

In [ ]:
mysql -u ${readonly_credentials[1]} -p${readonly_credentials[0]} -e "select user from mysql.user;"

Confirm that Bob's dynamic DB user can read from `my_app.customers` table.

In [ ]:
mysql -u ${readonly_credentials[1]} -p${readonly_credentials[0]} -e "select first_name from my_app.customers;"

Confirm that Bob's dynamic DB user can NOT write to `my_app.customers` table.

In [ ]:
mysql -u ${readonly_credentials[1]} -p${readonly_credentials[0]} -e "INSERT IGNORE into my_app.customers VALUES
  (10, '3/14/69', 'Moayad', 'Ismail', '2020-01-01T14:49:12.301977', '360-56-6750', 'Tyler, Texas', '7000000')"

## Managing Leases

#### List the Vault leases

In [ ]:
export VAULT_TOKEN=root

In [ ]:
vault list sys/leases/lookup/lob_a/db01/creds/db-user-readonly
keys=$(vault list -format=json sys/leases/lookup/lob_a/db01/creds/db-user-readonly | jq .[] -r)
export db_user_readonly_lease=($keys)

In [ ]:
vault list sys/leases/lookup/lob_a/db01/creds/db-user-readwrite
keys=$(vault list -format=json sys/leases/lookup/lob_a/db01/creds/db-user-readwrite | jq .[] -r)
export db_user_readwrite_lease=($keys)

#### Details on a lease, e.g. the remaining lifetime of the credential

In [ ]:
vault write sys/leases/lookup lease_id=lob_a/db01/creds/db-user-readwrite/${db_user_readwrite_lease[0]}

In [ ]:
vault write sys/leases/lookup lease_id=lob_a/db01/creds/db-user-readonly/${db_user_readonly_lease[0]}

#### Renew credentials

In [ ]:
vault write sys/leases/renew lease_id="lob_a/db01/creds/db-user-readwrite/${db_user_readwrite_lease[0]}"

#### Revoke all Vault leases

In [ ]:
vault lease revoke -prefix=true lob_a/db01/creds/db-user-readwrite
vault lease revoke -prefix=true lob_a/db01/creds/db-user-readonly

In [ ]:
vault list sys/leases/lookup/lob_a/db01/creds/db-user-readwrite
vault list sys/leases/lookup/lob_a/db01/creds/db-user-readonly

## Cleanup

#### Disable Database Secret Engine

In [ ]:
vault secrets disable lob_a/db01/ 

#### Disable Userpass Auth Method

In [ ]:
vault auth disable userpass

#### Shutdown Docker

In [ ]:
docker stop workshop-mysql
docker ps

### Remove artifacts

In [ ]:
rm -rf pov_db_dyn_cred.txt pov_db_static_cred.txt

## END